# Example of running sar-pre-processing package

## 1. Requirements

- Installation of sar-pre-processing package
- Installation of ESA's SNAP Toolbox (https://step.esa.int/main/download/snap-download/)
    - Update SNAP Toolbox after installing for fixing bugs within the software
    - SNAP Toolbox (Linux version) need libgfortran for specific operations but currently libgfortran is not installed during the installation process of SNAP therefore you might use
```sudo apt-get install gfortran```
- Sentinel-1 SLC data
    - location where files are stored (locally)<br>
    **or**
    - use 2. to download data from the internet
        - Free account for Copernicus Sentinel Data Hub needs to be created (https://scihub.copernicus.eu/dhus/#/self-registration)
        - free disk space needed
- For plotting purposes within this jupyter notebook matplotlib needs to be installed
    - ```conda install -c conda-forge matplotlib```

## 2. Download sample data from Sentinel Data Hub

### <span style="color:red">Create Account (https://scihub.copernicus.eu/dhus/#/self-registration) and change user and password below</span>.

In [1]:
# connect to the API
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
user = 'username'
password = 'password'
# initialize settings
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/apihub/')

In [ ]:
# search by polygon (MNI test site coordinates), time, and SciHub query keywords
footprint = geojson_to_wkt(read_geojson('coordinates_mni.geojson'))
products = api.query(footprint,
                     date=('20210101', '20210120'),
                     platformname='Sentinel-1',
                     producttype='SLC')
print('Following products will be downloaded')
print(api.to_dataframe(products).title.values)

print('These {} product need {} Gb of disk space'.format(len(products), api.get_products_size(products)))

**Start download process (<span style="color:red">Attention: might take a while and data will requries some free disk space</span>)**

In [2]:
# download all results from the search
# files will be downloaded to specified path
import os
path = "/path/to/data/"
try:
    os.makedirs(path)
except: FileExistsError
api.download_all(products, path)

NameError: name 'products' is not defined

## 3. Use sar-pre-processing package to process Sentinel-1 SLC data

Set paths for
- input_folder (location with stored Sentinel-1 SLC data)
- output_folder (location where processed data will be stored)
- gpt_loction (gpt is located in the bin folder of your SNAP installation)' 

In [3]:
input_folder = '/media/nas_data/Thomas/S1/data/Wallerfing_MNI'
output_folder = '/media/nas_data/Thomas/S1/processed/MNI_2017_new_final'
gpt_location = os.path.expanduser('~/snap/bin/gpt')

Create config file with information about input, output and gpt loction

In [4]:
import yaml

with open('sample_config_file.yaml') as stream:
   data = yaml.safe_load(stream)

data['input_folder'] = input_folder
data['output_folder'] = output_folder
data['gpt'] = gpt_location
data['year'] = '2017'
## Define region of interest
data['region']['lr']['lat'] = 48.1 # lower right latitude
data['region']['lr']['lon'] = 11.9 # lower right longitude
data['region']['ul']['lat'] = 48.4 # upper left latitude
data['region']['ul']['lon'] = 11.6 # upper left longitude

with open('test_config_file.yaml', 'wb') as stream:
   yaml.safe_dump(data, stream, default_flow_style=False, 
                  explicit_start=True, allow_unicode=True, encoding='utf-8')

Start pre-processing steps

In [5]:
from sar_pre_processing.sar_pre_processor import *
import warnings
warnings.filterwarnings("ignore")
processing = SARPreProcessor(config='test_config_file.yaml')
#file1, file2,file3,file4 = processing.file_list_new()

In [5]:
file44 = tuple([file1])
file117 = tuple([file2])
file95 = tuple([file3])
file168 = tuple([file4])


In [6]:
print(file44)


(['/media/nas_data/Thomas/S1/processed/MNI_2017_new_final/step2/S1A_IW_SLC__1SDV_20170805T165854_20170805T165921_017791_01DCFD_4C51', '/media/nas_data/Thomas/S1/processed/MNI_2017_new_final/step2/S1A_IW_SLC__1SDV_20171215T165855_20171215T165922_019716_021835_01FA', '/media/nas_data/Thomas/S1/processed/MNI_2017_new_final/step2/S1A_IW_SLC__1SDV_20170606T165851_20170606T165918_016916_01C25A_DCD8', '/media/nas_data/Thomas/S1/processed/MNI_2017_new_final/step2/S1A_IW_SLC__1SDV_20170218T165852_20170218T165919_015341_01927E_81A9', '/media/nas_data/Thomas/S1/processed/MNI_2017_new_final/step2/S1A_IW_SLC__1SDV_20171203T165856_20171203T165923_019541_0212C1_BCA0', '/media/nas_data/Thomas/S1/processed/MNI_2017_new_final/step2/S1B_IW_SLC__1SDV_20171022T165838_20171022T165905_007945_00E089_BE25', '/media/nas_data/Thomas/S1/processed/MNI_2017_new_final/step2/S1B_IW_SLC__1SDV_20170519T165809_20170519T165836_005670_009EE2_283D', '/media/nas_data/Thomas/S1/processed/MNI_2017_new_final/step2/S1A_IW_SLC__

In [8]:
processing.create_processing_file_list()

INFO:root:Found files within input folder: 845
INFO:root:Number of found files for year 2017: 265
INFO:root:Number of found files containing area of interest: 262
INFO:root:Number of found files that were double processed: 0.0
INFO:root:Number of found files with border issues: 84


(['/media/nas_data/Thomas/S1/data/Wallerfing_MNI/S1A_IW_SLC__1SDV_20170101T165853_20170101T165920_014641_017CE5_63E5.zip',
  '/media/nas_data/Thomas/S1/data/Wallerfing_MNI/S1A_IW_SLC__1SDV_20170105T051805_20170105T051832_014692_017E7C_66C7.zip',
  '/media/nas_data/Thomas/S1/data/Wallerfing_MNI/S1A_IW_SLC__1SDV_20170106T170705_20170106T170732_014714_017F1D_E474.zip',
  '/media/nas_data/Thomas/S1/data/Wallerfing_MNI/S1A_IW_SLC__1SDV_20170110T052613_20170110T052641_014765_0180A8_38BD.zip',
  '/media/nas_data/Thomas/S1/data/Wallerfing_MNI/S1A_IW_SLC__1SDV_20170113T165853_20170113T165920_014816_018237_22BE.zip',
  '/media/nas_data/Thomas/S1/data/Wallerfing_MNI/S1A_IW_SLC__1SDV_20170118T170704_20170118T170731_014889_018490_9D8E.zip',
  '/media/nas_data/Thomas/S1/data/Wallerfing_MNI/S1A_IW_SLC__1SDV_20170125T165853_20170125T165920_014991_0187A4_EFC5.zip',
  '/media/nas_data/Thomas/S1/data/Wallerfing_MNI/S1A_IW_SLC__1SDV_20170129T051804_20170129T051832_015042_018943_C457.zip',
  '/media/nas_da

In [12]:
processing.file_list = file44

In [13]:
processing.pre_process_step3()

INFO:ComponentProgress:0
INFO:ComponentProgress:0
INFO:root:0
INFO:root:2021-04-22 14:39:54.888903
INFO:ComponentProgress:1
INFO:ComponentProgress:1
INFO:root:0
INFO:root:2021-04-22 14:41:44.119696
INFO:ComponentProgress:3
INFO:ComponentProgress:3
INFO:root:0
INFO:root:2021-04-22 14:43:31.407049
INFO:ComponentProgress:5
INFO:ComponentProgress:5
INFO:root:0
INFO:root:2021-04-22 14:45:19.096878
INFO:ComponentProgress:7
INFO:ComponentProgress:7
INFO:root:0
INFO:root:2021-04-22 14:47:08.174383
INFO:ComponentProgress:9
INFO:ComponentProgress:9
INFO:root:0
INFO:root:2021-04-22 14:48:58.713611
INFO:ComponentProgress:10
INFO:ComponentProgress:10
INFO:root:0
INFO:root:2021-04-22 14:50:52.097086
INFO:ComponentProgress:12
INFO:ComponentProgress:12
INFO:root:0
INFO:root:2021-04-22 14:52:43.315349
INFO:ComponentProgress:14
INFO:ComponentProgress:14
INFO:root:0
INFO:root:2021-04-22 14:54:34.398466
INFO:ComponentProgress:16
INFO:ComponentProgress:16
INFO:root:0
INFO:root:2021-04-22 14:56:23.452913
IN

In [14]:
processing.file_list = file117
processing.pre_process_step3()

INFO:ComponentProgress:0
INFO:ComponentProgress:0
INFO:root:0
INFO:root:2021-04-22 16:24:06.142200
INFO:ComponentProgress:1
INFO:ComponentProgress:1
INFO:root:0
INFO:root:2021-04-22 16:25:56.981749
INFO:ComponentProgress:3
INFO:ComponentProgress:3
INFO:root:0
INFO:root:2021-04-22 16:27:50.520556
INFO:ComponentProgress:5
INFO:ComponentProgress:5
INFO:root:0
INFO:root:2021-04-22 16:29:38.671481
INFO:ComponentProgress:7
INFO:ComponentProgress:7
INFO:root:0
INFO:root:2021-04-22 16:31:30.982120
INFO:ComponentProgress:9
INFO:ComponentProgress:9
INFO:root:0
INFO:root:2021-04-22 16:33:29.147688
INFO:ComponentProgress:10
INFO:ComponentProgress:10
INFO:root:0
INFO:root:2021-04-22 16:35:19.596499
INFO:ComponentProgress:12
INFO:ComponentProgress:12
INFO:root:0
INFO:root:2021-04-22 16:37:40.617877
INFO:ComponentProgress:14
INFO:ComponentProgress:14
INFO:root:0
INFO:root:2021-04-22 16:40:24.768299
INFO:ComponentProgress:16
INFO:ComponentProgress:16
INFO:root:0
INFO:root:2021-04-22 16:42:32.757141
IN

In [15]:
processing.file_list = file95
processing.pre_process_step3()

INFO:ComponentProgress:0
INFO:ComponentProgress:0
INFO:root:0
INFO:root:2021-04-22 18:11:34.508231
INFO:ComponentProgress:1
INFO:ComponentProgress:1
INFO:root:0
INFO:root:2021-04-22 18:13:47.955735
INFO:ComponentProgress:3
INFO:ComponentProgress:3
INFO:root:0
INFO:root:2021-04-22 18:15:35.755769
INFO:ComponentProgress:5
INFO:ComponentProgress:5
INFO:root:0
INFO:root:2021-04-22 18:17:21.681182
INFO:ComponentProgress:7
INFO:ComponentProgress:7
INFO:root:0
INFO:root:2021-04-22 18:19:07.467390
INFO:ComponentProgress:8
INFO:ComponentProgress:8
INFO:root:0
INFO:root:2021-04-22 18:20:53.529031
INFO:ComponentProgress:10
INFO:ComponentProgress:10
INFO:root:0
INFO:root:2021-04-22 18:22:41.675048
INFO:ComponentProgress:12
INFO:ComponentProgress:12
INFO:root:0
INFO:root:2021-04-22 18:24:27.976080
INFO:ComponentProgress:14
INFO:ComponentProgress:14
INFO:root:0
INFO:root:2021-04-22 18:26:15.688656
INFO:ComponentProgress:15
INFO:ComponentProgress:15
INFO:root:0
INFO:root:2021-04-22 18:28:01.307581
IN

In [16]:
processing.file_list = file168
processing.pre_process_step3()

INFO:ComponentProgress:0
INFO:ComponentProgress:0
INFO:root:0
INFO:root:2021-04-22 19:54:01.661117
INFO:ComponentProgress:1
INFO:ComponentProgress:1
INFO:root:0
INFO:root:2021-04-22 19:55:47.616832
INFO:ComponentProgress:3
INFO:ComponentProgress:3
INFO:root:0
INFO:root:2021-04-22 19:57:32.948638
INFO:ComponentProgress:5
INFO:ComponentProgress:5
INFO:root:0
INFO:root:2021-04-22 19:59:18.386957
INFO:ComponentProgress:7
INFO:ComponentProgress:7
INFO:root:0
INFO:root:2021-04-22 20:01:04.629441
INFO:ComponentProgress:9
INFO:ComponentProgress:9
INFO:root:0
INFO:root:2021-04-22 20:02:54.608515
INFO:ComponentProgress:11
INFO:ComponentProgress:11
INFO:root:0
INFO:root:2021-04-22 20:04:56.584665
INFO:ComponentProgress:13
INFO:ComponentProgress:13
INFO:root:0
INFO:root:2021-04-22 20:06:46.267956
INFO:ComponentProgress:15
INFO:ComponentProgress:15
INFO:root:0
INFO:root:2021-04-22 20:08:30.660816
INFO:ComponentProgress:16
INFO:ComponentProgress:16
INFO:root:0
INFO:root:2021-04-22 20:10:15.510235
IN

In [6]:
processing.create_processing_file_list()
print('start add netcdf information')
processing.add_netcdf_information()
print('start create netcdf stack')
processing.create_netcdf_stack()

INFO:root:Found files within input folder: 845
INFO:root:Number of found files for year 2017: 265
INFO:root:area of interest not specified
INFO:root:Number of found files that were double processed: 0.0
INFO:root:Number of found files with border issues: 90


start add netcdf information
/media/nas_data/Thomas/S1/processed/MNI_2017_new_final/step3/S1A_IW_SLC__1SDV_20170907T052608_20170907T052635_018265_01EB5E_334F_GC_RC_No_Su_Co_speckle.nc
> /home/tweiss/.local/lib/python3.9/site-packages/multiply_sar_pre_processing-0.1-py3.9.egg/sar_pre_processing/sar_pre_processor.py(532)add_netcdf_information()

ipdb> c


RuntimeError: NetCDF: Can't open HDF5 attribute

In [16]:
import subprocess
subprocess.call(os.path.join('/media/tweiss/Work/GIT/GitHub/multiply-org/sar-pre-processing/sar_pre_processing', 'solve_projection_problem.sh ' + processing.config.output_folder_step3), shell=True)

0

In [10]:
import subprocess

In [15]:
os.path.join('/media/tweiss/Work/GIT/GitHub/multiply-org/sar-pre-processing/sar_pre_processing', 'solve_projection_problem.sh ' + processing.config.output_folder_step3)

'/media/tweiss/Work/GIT/GitHub/multiply-org/sar-pre-processing/sar_pre_processing/solve_projection_problem.sh /media/nas_data/Thomas/S1/processed/MNI_2017_new_final/step3'

In [ ]:
from sar_pre_processing.sar_pre_processor import *
import warnings
warnings.filterwarnings("ignore")

processing = SARPreProcessor(config='test_config_file.yaml')
processing.create_processing_file_list()
print('start step 1')
processing.pre_process_step1()
print('start step 2')
processing.pre_process_step2()
print('start step 3')
processing.pre_process_step3()
print('start add netcdf information')
processing.add_netcdf_information()
print('start create netcdf stack')
processing.create_netcdf_stack()

## 3. View processed data

Load netcdf file with processed data

In [ ]:
import os
print(os.getcwd())
print(output_folder)

In [ ]:
from netCDF4 import Dataset
import numpy as np

my_example_nc_file = os.path.join(output_folder, 'data.nc')
data = Dataset(my_example_nc_file, mode='r') 

View information about dataset

In [ ]:
data

Read data from netcdf file

In [ ]:
data.variables['orbitdirection'][:]
data.variables['time'][:]
lons = data.variables['lon'][:]
lats = data.variables['lat'][:]
vv = data.variables['sigma0_vv_multi'][:]

vv_units = data.variables['sigma0_vv_multi'].units

Close netcdf file

In [ ]:
data.close()

Plot vv polorized data

In [ ]:
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def f(x):
    # Problem: border pixel might be zero or negative
    # pixel eqal or smaller than zero are set to nan
    array = np.copy(vv[x])
    array[array <= 0] = np.nan
    # plot backscatter data in dB scale
    plt.imshow(10*np.log10(array))
    cbar = plt.colorbar()
    cbar.set_label('dB')
    plt.clim(-25, 0)

interactive_plot = interactive(f, x=(0,len(vv)-1))
interactive_plot